### creating a spark instance

In [132]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("date file").getOrCreate()

### reading csv file

In [133]:
ddc=spark.read.csv("DailyDelhiClimateTest.csv",header=True,inferSchema=True)

In [134]:
ddc.show()

+----------+------------------+-----------------+------------------+------------------+
|      date|          meantemp|         humidity|        wind_speed|      meanpressure|
+----------+------------------+-----------------+------------------+------------------+
|2017-01-01| 15.91304347826087| 85.8695652173913| 2.743478260869565|              59.0|
|2017-01-02|              18.5|77.22222222222223|2.8944444444444444|1018.2777777777778|
|2017-01-03| 17.11111111111111|81.88888888888889| 4.016666666666667|1018.3333333333334|
|2017-01-04|              18.7|            70.05|             4.545|            1015.7|
|2017-01-05| 18.38888888888889|74.94444444444444|3.3000000000000003|1014.3333333333334|
|2017-01-06|19.318181818181817|79.31818181818181| 8.681818181818182|1011.7727272727273|
|2017-01-07|14.708333333333334|95.83333333333333|10.041666666666664|          1011.375|
|2017-01-08| 15.68421052631579|83.52631578947368|              1.95|           1015.55|
|2017-01-09|14.571428571428571|8

In [135]:
ddc.dtypes

[('date', 'string'),
 ('meantemp', 'double'),
 ('humidity', 'double'),
 ('wind_speed', 'double'),
 ('meanpressure', 'double')]

In [136]:
ddc.describe()

DataFrame[summary: string, date: string, meantemp: string, humidity: string, wind_speed: string, meanpressure: string]

In [137]:
ddc.columns

['date', 'meantemp', 'humidity', 'wind_speed', 'meanpressure']

In [138]:
ddc.select('date').describe().show()

+-------+----------+
|summary|      date|
+-------+----------+
|  count|       114|
|   mean|      null|
| stddev|      null|
|    min|2017-01-01|
|    max|2017-04-24|
+-------+----------+



In [139]:
ddc.describe().show()

+-------+----------+-----------------+------------------+------------------+------------------+
|summary|      date|         meantemp|          humidity|        wind_speed|      meanpressure|
+-------+----------+-----------------+------------------+------------------+------------------+
|  count|       114|              114|               114|               114|               114|
|   mean|      null|21.71307892022824|56.258361837263266| 8.143924054573581|  1004.03508969052|
| stddev|      null|6.360071848767031|19.068082852306677|3.5880494654730275| 89.47469204777897|
|    min|2017-01-01|             11.0|             17.75|1.3875000000000002|              59.0|
|    max|2017-04-24|             34.5| 95.83333333333333|19.314285714285713|1022.8095238095239|
+-------+----------+-----------------+------------------+------------------+------------------+



In [140]:
ddc.describe('date').show()

+-------+----------+
|summary|      date|
+-------+----------+
|  count|       114|
|   mean|      null|
| stddev|      null|
|    min|2017-01-01|
|    max|2017-04-24|
+-------+----------+



In [141]:
ddc.printSchema()

root
 |-- date: string (nullable = true)
 |-- meantemp: double (nullable = true)
 |-- humidity: double (nullable = true)
 |-- wind_speed: double (nullable = true)
 |-- meanpressure: double (nullable = true)



### checking column wise count of nulls and nas in dataset

In [142]:
from pyspark.sql.functions import count,when,col,isnan
ddc.select([count(when(col(c).isNull(),c)).alias(c) for c in ddc.columns]).show()

+----+--------+--------+----------+------------+
|date|meantemp|humidity|wind_speed|meanpressure|
+----+--------+--------+----------+------------+
|   0|       0|       0|         0|           0|
+----+--------+--------+----------+------------+



In [143]:
ddc.select([count(when(isnan(c),c)).alias(c) for c in ddc.columns]).show()

+----+--------+--------+----------+------------+
|date|meantemp|humidity|wind_speed|meanpressure|
+----+--------+--------+----------+------------+
|   0|       0|       0|         0|           0|
+----+--------+--------+----------+------------+



In [144]:
type(ddc)

pyspark.sql.dataframe.DataFrame

In [145]:
ddc.date

Column<b'date'>

In [146]:
ddc['date']

Column<b'date'>

In [147]:
ddc.select('date')

DataFrame[date: string]

In [148]:
ddc.select('date').show(5)

+----------+
|      date|
+----------+
|2017-01-01|
|2017-01-02|
|2017-01-03|
|2017-01-04|
|2017-01-05|
+----------+
only showing top 5 rows



In [149]:
ddc.select('date','meantemp').show(5)

+----------+-----------------+
|      date|         meantemp|
+----------+-----------------+
|2017-01-01|15.91304347826087|
|2017-01-02|             18.5|
|2017-01-03|17.11111111111111|
|2017-01-04|             18.7|
|2017-01-05|18.38888888888889|
+----------+-----------------+
only showing top 5 rows



In [150]:
from pyspark.sql.functions import *
ddc.select(round(ddc['meantemp'],2).alias('mean temperature')).show(5)

+----------------+
|mean temperature|
+----------------+
|           15.91|
|            18.5|
|           17.11|
|            18.7|
|           18.39|
+----------------+
only showing top 5 rows



### obtaining mean value of humidity

In [151]:
x=ddc.select(avg(ddc.humidity).alias('mean humidity')).collect()
x[0][0]

56.258361837263266

In [152]:
ddc.select(max(ddc.wind_speed).alias('maximum wind speed')).show()

+------------------+
|maximum wind speed|
+------------------+
|19.314285714285713|
+------------------+



### finding humidity to wind speed ratio

In [153]:
ddc1=ddc.withColumn('hum_to_speed',round((ddc.humidity/ddc.wind_speed),2))
ddc1.show(5)

+----------+-----------------+-----------------+------------------+------------------+------------+
|      date|         meantemp|         humidity|        wind_speed|      meanpressure|hum_to_speed|
+----------+-----------------+-----------------+------------------+------------------+------------+
|2017-01-01|15.91304347826087| 85.8695652173913| 2.743478260869565|              59.0|        31.3|
|2017-01-02|             18.5|77.22222222222223|2.8944444444444444|1018.2777777777778|       26.68|
|2017-01-03|17.11111111111111|81.88888888888889| 4.016666666666667|1018.3333333333334|       20.39|
|2017-01-04|             18.7|            70.05|             4.545|            1015.7|       15.41|
|2017-01-05|18.38888888888889|74.94444444444444|3.3000000000000003|1014.3333333333334|       22.71|
+----------+-----------------+-----------------+------------------+------------------+------------+
only showing top 5 rows



In [154]:
type(ddc1)

pyspark.sql.dataframe.DataFrame

### creating a copy column of existing column

In [155]:
ddc1 = ddc1.withColumn('mean temperature',ddc.meantemp)
ddc1.show(5)

+----------+-----------------+-----------------+------------------+------------------+------------+-----------------+
|      date|         meantemp|         humidity|        wind_speed|      meanpressure|hum_to_speed| mean temperature|
+----------+-----------------+-----------------+------------------+------------------+------------+-----------------+
|2017-01-01|15.91304347826087| 85.8695652173913| 2.743478260869565|              59.0|        31.3|15.91304347826087|
|2017-01-02|             18.5|77.22222222222223|2.8944444444444444|1018.2777777777778|       26.68|             18.5|
|2017-01-03|17.11111111111111|81.88888888888889| 4.016666666666667|1018.3333333333334|       20.39|17.11111111111111|
|2017-01-04|             18.7|            70.05|             4.545|            1015.7|       15.41|             18.7|
|2017-01-05|18.38888888888889|74.94444444444444|3.3000000000000003|1014.3333333333334|       22.71|18.38888888888889|
+----------+-----------------+-----------------+--------

In [156]:
ddc1.dtypes

[('date', 'string'),
 ('meantemp', 'double'),
 ('humidity', 'double'),
 ('wind_speed', 'double'),
 ('meanpressure', 'double'),
 ('hum_to_speed', 'double'),
 ('mean temperature', 'double')]

### renaming column name

In [157]:
ddc1=ddc1.withColumnRenamed('mean temperature','mean_temperature')
ddc1.show()

+----------+------------------+-----------------+------------------+------------------+------------+------------------+
|      date|          meantemp|         humidity|        wind_speed|      meanpressure|hum_to_speed|  mean_temperature|
+----------+------------------+-----------------+------------------+------------------+------------+------------------+
|2017-01-01| 15.91304347826087| 85.8695652173913| 2.743478260869565|              59.0|        31.3| 15.91304347826087|
|2017-01-02|              18.5|77.22222222222223|2.8944444444444444|1018.2777777777778|       26.68|              18.5|
|2017-01-03| 17.11111111111111|81.88888888888889| 4.016666666666667|1018.3333333333334|       20.39| 17.11111111111111|
|2017-01-04|              18.7|            70.05|             4.545|            1015.7|       15.41|              18.7|
|2017-01-05| 18.38888888888889|74.94444444444444|3.3000000000000003|1014.3333333333334|       22.71| 18.38888888888889|
|2017-01-06|19.318181818181817|79.318181

### filtering data based on a condition and obtaining row corresponding to the condition

In [158]:
res=ddc1.filter(ddc1.date=='2017-01-20').collect()

In [159]:
result=res[0]

In [160]:
result.asDict()

{'date': '2017-01-20',
 'meantemp': 15.263157894736842,
 'humidity': 66.47368421052632,
 'wind_speed': 11.226315789473684,
 'meanpressure': 1021.7894736842105,
 'hum_to_speed': 5.92,
 'mean_temperature': 15.263157894736842}

In [161]:
for i in result:
    print(i)

2017-01-20
15.263157894736842
66.47368421052632
11.226315789473684
1021.7894736842105
5.92
15.263157894736842


### using multiple filter conditions

In [162]:
ddc1.filter((ddc1['meanpressure']>50) & ~(ddc1['wind_speed']<15)).select(ddc1['date']).show()

+----------+
|      date|
+----------+
|2017-02-24|
|2017-03-26|
|2017-04-10|
|2017-04-11|
+----------+



### creating view from dataframe

In [163]:
ddc1.createOrReplaceTempView("DL_climate")

In [164]:
type(ddc1)

pyspark.sql.dataframe.DataFrame

In [165]:
q1=spark.sql("select * from DL_climate")
q1.show(2)

+----------+-----------------+-----------------+------------------+------------------+------------+-----------------+
|      date|         meantemp|         humidity|        wind_speed|      meanpressure|hum_to_speed| mean_temperature|
+----------+-----------------+-----------------+------------------+------------------+------------+-----------------+
|2017-01-01|15.91304347826087| 85.8695652173913| 2.743478260869565|              59.0|        31.3|15.91304347826087|
|2017-01-02|             18.5|77.22222222222223|2.8944444444444444|1018.2777777777778|       26.68|             18.5|
+----------+-----------------+-----------------+------------------+------------------+------------+-----------------+
only showing top 2 rows



In [166]:
q2=spark.sql("select date,round(wind_speed,2) as wind_speed,round(meanpressure,2) as meanpressure from dl_climate where humidity between 50 and 60")
q2.show()

+----------+----------+------------+
|      date|wind_speed|meanpressure|
+----------+----------+------------+
|2017-02-16|     10.44|     1017.63|
|2017-02-22|      7.41|     1012.25|
|2017-02-25|     11.93|     1014.29|
|2017-02-26|     11.11|     1012.38|
|2017-02-27|       5.1|     1014.25|
|2017-02-28|      3.91|     1013.11|
|2017-03-02|      5.93|     1012.15|
|2017-03-08|     10.26|     1008.91|
|2017-03-12|      7.64|     1014.75|
|2017-03-14|      4.52|     1016.56|
|2017-03-15|      7.18|     1014.13|
|2017-03-17|     11.49|      1010.7|
|2017-03-19|      9.16|     1011.89|
|2017-03-20|     10.08|     1012.56|
+----------+----------+------------+



In [167]:
q3=spark.sql("select distinct month(date) from dl_climate order by 1")
q3.show()

+-------------------------+
|month(CAST(date AS DATE))|
+-------------------------+
|                        1|
|                        2|
|                        3|
|                        4|
+-------------------------+



In [179]:
q4=spark.sql("select * from dl_climate where month(date)==4")
q4.filter(q4.meanpressure<1000).select(['date','hum_to_speed','wind_speed']).show()

+----------+------------+----------+
|      date|hum_to_speed|wind_speed|
+----------+------------+----------+
|2017-04-20|        4.94|    5.5625|
|2017-04-21|        5.66|    6.9625|
+----------+------------+----------+



### analysing data based on date

In [188]:
ddc1.select(dayofmonth(ddc1.date)).distinct().orderBy('date').count()

31

In [190]:
ddc1.select(month(ddc1.date)).count()

114

In [193]:
ddc1.select(month(ddc1.date)).show(10)

+-----------+
|month(date)|
+-----------+
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
+-----------+
only showing top 10 rows



In [195]:
ddc1.select(dayofyear(ddc1.date)).distinct().show()

+---------------+
|dayofyear(date)|
+---------------+
|             31|
|             85|
|             65|
|             53|
|             78|
|            108|
|             34|
|            101|
|             81|
|             28|
|             76|
|             26|
|             27|
|             44|
|            103|
|             12|
|             91|
|             22|
|             93|
|            111|
+---------------+
only showing top 20 rows



In [197]:
ddc1.select(year(ddc1.date)).distinct().show()

+----------+
|year(date)|
+----------+
|      2017|
+----------+



In [198]:
ddc1=ddc1.withColumn('month',month(ddc1.date))
ddc1.show(3)

+----------+-----------------+-----------------+------------------+------------------+------------+-----------------+-----+
|      date|         meantemp|         humidity|        wind_speed|      meanpressure|hum_to_speed| mean_temperature|month|
+----------+-----------------+-----------------+------------------+------------------+------------+-----------------+-----+
|2017-01-01|15.91304347826087| 85.8695652173913| 2.743478260869565|              59.0|        31.3|15.91304347826087|    1|
|2017-01-02|             18.5|77.22222222222223|2.8944444444444444|1018.2777777777778|       26.68|             18.5|    1|
|2017-01-03|17.11111111111111|81.88888888888889| 4.016666666666667|1018.3333333333334|       20.39|17.11111111111111|    1|
+----------+-----------------+-----------------+------------------+------------------+------------+-----------------+-----+
only showing top 3 rows



In [200]:
ddc1=ddc1.withColumn('day',dayofmonth(ddc1.date))
ddc1.show(5)

+----------+-----------------+-----------------+------------------+------------------+------------+-----------------+-----+---+
|      date|         meantemp|         humidity|        wind_speed|      meanpressure|hum_to_speed| mean_temperature|month|day|
+----------+-----------------+-----------------+------------------+------------------+------------+-----------------+-----+---+
|2017-01-01|15.91304347826087| 85.8695652173913| 2.743478260869565|              59.0|        31.3|15.91304347826087|    1|  1|
|2017-01-02|             18.5|77.22222222222223|2.8944444444444444|1018.2777777777778|       26.68|             18.5|    1|  2|
|2017-01-03|17.11111111111111|81.88888888888889| 4.016666666666667|1018.3333333333334|       20.39|17.11111111111111|    1|  3|
|2017-01-04|             18.7|            70.05|             4.545|            1015.7|       15.41|             18.7|    1|  4|
|2017-01-05|18.38888888888889|74.94444444444444|3.3000000000000003|1014.3333333333334|       22.71|18.38

In [204]:
rs=ddc1.groupBy("month").agg({'meantemp':'max'})
rs=rs.withColumnRenamed('max(meantemp)','maximum mean temperature')
rs.show(5)

+-----+------------------------+
|month|maximum mean temperature|
+-----+------------------------+
|    1|                    21.0|
|    3|                    31.0|
|    4|                    34.5|
|    2|                  23.375|
+-----+------------------------+



In [205]:
ddc1.select(weekofyear(ddc1.date)).show()

+----------------+
|weekofyear(date)|
+----------------+
|              52|
|               1|
|               1|
|               1|
|               1|
|               1|
|               1|
|               1|
|               2|
|               2|
|               2|
|               2|
|               2|
|               2|
|               2|
|               3|
|               3|
|               3|
|               3|
|               3|
+----------------+
only showing top 20 rows



In [214]:
ddc1=ddc1.withColumn('week_num',weekofyear(ddc1.date))
ddc1.show(5)

+----------+-----------------+-----------------+------------------+------------------+------------+-----------------+-----+---+--------+
|      date|         meantemp|         humidity|        wind_speed|      meanpressure|hum_to_speed| mean_temperature|month|day|week_num|
+----------+-----------------+-----------------+------------------+------------------+------------+-----------------+-----+---+--------+
|2017-01-01|15.91304347826087| 85.8695652173913| 2.743478260869565|              59.0|        31.3|15.91304347826087|    1|  1|      52|
|2017-01-02|             18.5|77.22222222222223|2.8944444444444444|1018.2777777777778|       26.68|             18.5|    1|  2|       1|
|2017-01-03|17.11111111111111|81.88888888888889| 4.016666666666667|1018.3333333333334|       20.39|17.11111111111111|    1|  3|       1|
|2017-01-04|             18.7|            70.05|             4.545|            1015.7|       15.41|             18.7|    1|  4|       1|
|2017-01-05|18.38888888888889|74.94444444

In [213]:
ddc1.select('week_num').distinct().show()

+--------+
|week_num|
+--------+
|      12|
|       1|
|      52|
|      13|
|       6|
|      16|
|       3|
|       5|
|      15|
|       9|
|      17|
|       4|
|       8|
|       7|
|      10|
|      11|
|      14|
|       2|
+--------+



In [212]:
ddc1.groupBy('week_num').agg(min(col('meantemp')).alias('minimum mean temperature'),min(col('wind_speed')).alias('minimum wind speed'),min(col('humidity')).alias('minimum humidity'),min(col('meanpressure')).alias('minimum mean pressure')).orderBy('week_num',ascending=True).show()

+--------+------------------------+------------------+------------------+---------------------+
|week_num|minimum mean temperature|minimum wind speed|  minimum humidity|minimum mean pressure|
+--------+------------------------+------------------+------------------+---------------------+
|       1|      14.708333333333334|              1.95|             70.05|             1011.375|
|       2|                    11.0| 3.630434782608696| 67.05882352941177|    1015.952380952381|
|       3|      11.722222222222221|5.8679999999999986| 66.47368421052632|              1017.15|
|       4|      14.863636363636363|3.3913043478260865|  68.1304347826087|   1017.3888888888889|
|       5|                   15.25|1.8545454545454545| 66.54545454545455|   1014.2105263157895|
|       6|      14.666666666666666|             1.625|             63.75|               1015.6|
|       7|      16.333333333333332| 5.557142857142857|             56.75|               1013.0|
|       8|                  18.625| 6.93